<a href="https://colab.research.google.com/github/huynqcharles/Predict-EPL-Matches-using-Neural-Network/blob/main/Predict-EPL-Matches-using-Neural-Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT LIBRARIES

In [ ]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
import kerastuner as kt

<ipython-input-2-dca03a43038f>:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


# LOAD THE DATA FROM CSV FILES

In [ ]:
# Read datas from the CSV into dataframes
folder='/content/'
raw_data = pd.read_csv(folder +'matches_before_2034-24.csv', encoding='latin-1')
test_data_2034_24 = pd.read_csv(folder +'Season 2023-2024.csv', encoding='latin-1')

<ipython-input-3-fb34dfd2e248>:3: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(folder +'matches_before_2034-24.csv', encoding='latin-1')




```
Key to results data:

Div = League Division
Date = Match Date (dd/mm/yy)
Time = Time of match kick off
HomeTeam = Home Team
AwayTeam = Away Team
FTHG and HG = Full Time Home Team Goals
FTAG and AG = Full Time Away Team Goals
FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
HTHG = Half Time Home Team Goals
HTAG = Half Time Away Team Goals
HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

Match Statistics (where available)
Attendance = Crowd Attendance
Referee = Match Referee
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
HHW = Home Team Hit Woodwork
AHW = Away Team Hit Woodwork
HC = Home Team Corners
AC = Away Team Corners
HF = Home Team Fouls Committed
AF = Away Team Fouls Committed
HFKC = Home Team Free Kicks Conceded
AFKC = Away Team Free Kicks Conceded
HO = Home Team Offsides
AO = Away Team Offsides
HY = Home Team Yellow Cards
AY = Away Team Yellow Cards
HR = Home Team Red Cards
AR = Away Team Red Cards
HBP = Home Team Bookings Points (10 = yellow, 25 = red)
ABP = Away Team Bookings Points (10 = yellow, 25 = red)

Note that Free Kicks Conceeded includes fouls, offsides and any other offense commmitted and will always be equal to or higher than the number of fouls. Fouls make up the vast majority of Free Kicks Conceded. Free Kicks Conceded are shown when specific data on Fouls are not available (France 2nd, Belgium 1st and Greece 1st divisions).

Note also that English and Scottish yellow cards do not include the initial yellow card when a second is shown to a player converting it into a red, but this is included as a yellow (plus red) for European games.


Key to 1X2 (match) betting odds data:

B365H = Bet365 home win odds
B365D = Bet365 draw odds
B365A = Bet365 away win odds
BSH = Blue Square home win odds
BSD = Blue Square draw odds
BSA = Blue Square away win odds
BWH = Bet&amp;Win home win odds
BWD = Bet&amp;Win draw odds
BWA = Bet&amp;Win away win odds
GBH = Gamebookers home win odds
GBD = Gamebookers draw odds
GBA = Gamebookers away win odds
IWH = Interwetten home win odds
IWD = Interwetten draw odds
IWA = Interwetten away win odds
LBH = Ladbrokes home win odds
LBD = Ladbrokes draw odds
LBA = Ladbrokes away win odds
PSH and PH = Pinnacle home win odds
PSD and PD = Pinnacle draw odds
PSA and PA = Pinnacle away win odds
SOH = Sporting Odds home win odds
SOD = Sporting Odds draw odds
SOA = Sporting Odds away win odds
SBH = Sportingbet home win odds
SBD = Sportingbet draw odds
SBA = Sportingbet away win odds
SJH = Stan James home win odds
SJD = Stan James draw odds
SJA = Stan James away win odds
SYH = Stanleybet home win odds
SYD = Stanleybet draw odds
SYA = Stanleybet away win odds
VCH = VC Bet home win odds
VCD = VC Bet draw odds
VCA = VC Bet away win odds
WHH = William Hill home win odds
WHD = William Hill draw odds
WHA = William Hill away win odds

Bb1X2 = Number of BetBrain bookmakers used to calculate match odds averages and maximums
BbMxH = Betbrain maximum home win odds
BbAvH = Betbrain average home win odds
BbMxD = Betbrain maximum draw odds
BbAvD = Betbrain average draw win odds
BbMxA = Betbrain maximum away win odds
BbAvA = Betbrain average away win odds

MaxH = Market maximum home win odds
MaxD = Market maximum draw win odds
MaxA = Market maximum away win odds
AvgH = Market average home win odds
AvgD = Market average draw win odds
AvgA = Market average away win odds



Key to total goals betting odds:

BbOU = Number of BetBrain bookmakers used to calculate over/under 2.5 goals (total goals) averages and maximums
BbMx&gt;2.5 = Betbrain maximum over 2.5 goals
BbAv&gt;2.5 = Betbrain average over 2.5 goals
BbMx&lt;2.5 = Betbrain maximum under 2.5 goals
BbAv&lt;2.5 = Betbrain average under 2.5 goals

GB&gt;2.5 = Gamebookers over 2.5 goals
GB&lt;2.5 = Gamebookers under 2.5 goals
B365&gt;2.5 = Bet365 over 2.5 goals
B365&lt;2.5 = Bet365 under 2.5 goals
P&gt;2.5 = Pinnacle over 2.5 goals
P&lt;2.5 = Pinnacle under 2.5 goals
Max&gt;2.5 = Market maximum over 2.5 goals
Max&lt;2.5 = Market maximum under 2.5 goals
Avg&gt;2.5 = Market average over 2.5 goals
Avg&lt;2.5 = Market average under 2.5 goals



Key to Asian handicap betting odds:

BbAH = Number of BetBrain bookmakers used to Asian handicap averages and maximums
BbAHh = Betbrain size of handicap (home team)
AHh = Market size of handicap (home team) (since 2019/2020)
BbMxAHH = Betbrain maximum Asian handicap home team odds
BbAvAHH = Betbrain average Asian handicap home team odds
BbMxAHA = Betbrain maximum Asian handicap away team odds
BbAvAHA = Betbrain average Asian handicap away team odds

GBAHH = Gamebookers Asian handicap home team odds
GBAHA = Gamebookers Asian handicap away team odds
GBAH = Gamebookers size of handicap (home team)
LBAHH = Ladbrokes Asian handicap home team odds
LBAHA = Ladbrokes Asian handicap away team odds
LBAH = Ladbrokes size of handicap (home team)
B365AHH = Bet365 Asian handicap home team odds
B365AHA = Bet365 Asian handicap away team odds
B365AH = Bet365 size of handicap (home team)
PAHH = Pinnacle Asian handicap home team odds
PAHA = Pinnacle Asian handicap away team odds
MaxAHH = Market maximum Asian handicap home team odds
MaxAHA = Market maximum Asian handicap away team odds
AvgAHH = Market average Asian handicap home team odds
AvgAHA = Market average Asian handicap away team odds
```



In [ ]:
raw_data.tail(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
7591,E0,28/05/2023,Arsenal,Wolves,5.0,0.0,H,3.0,0.0,H,...,3.09,-1.75,1.93,1.97,1.96,1.93,2.02,2.02,1.93,1.92
7592,E0,28/05/2023,Aston Villa,Brighton,2.0,1.0,H,2.0,1.0,H,...,2.62,-0.50,1.89,2.01,1.88,2.03,1.95,2.08,1.87,2.00
7593,E0,28/05/2023,Brentford,Man City,1.0,0.0,H,0.0,0.0,D,...,2.01,0.25,1.93,1.97,2.05,1.86,2.28,1.97,2.01,1.85
7594,E0,28/05/2023,Chelsea,Newcastle,1.0,1.0,D,1.0,1.0,D,...,2.51,0.25,1.70,2.10,1.77,2.15,1.99,2.21,1.80,2.06
7595,E0,28/05/2023,Crystal Palace,Nott'm Forest,1.0,1.0,D,0.0,1.0,A,...,2.03,-1.00,2.06,1.84,2.06,1.86,2.12,1.88,2.04,1.83
7596,E0,28/05/2023,Everton,Bournemouth,1.0,0.0,H,0.0,0.0,D,...,2.14,-1.00,2.02,1.77,2.10,1.81,2.17,1.92,2.03,1.83
7597,E0,28/05/2023,Leeds,Tottenham,1.0,4.0,A,0.0,1.0,A,...,2.50,0.25,1.84,2.06,1.83,2.10,1.90,2.14,1.81,2.06
7598,E0,28/05/2023,Leicester,West Ham,2.0,1.0,H,1.0,0.0,H,...,2.51,-0.25,1.75,2.05,1.85,2.06,1.90,2.16,1.82,2.04
7599,E0,28/05/2023,Man United,Fulham,2.0,1.0,H,1.0,1.0,D,...,2.95,-1.25,1.98,1.92,1.98,1.93,2.07,1.98,1.97,1.89
7600,E0,28/05/2023,Southampton,Liverpool,4.0,4.0,D,2.0,2.0,D,...,3.22,1.25,1.82,2.08,1.85,2.07,1.96,2.12,1.88,1.98


# PREPROCESSING THE DATA

In [ ]:
dataset = raw_data.drop(["Div"], axis=1)
testset_2023_2024 = test_data_2034_24.drop(["Div"], axis=1)

if testset_2023_2024.shape[1] < dataset.shape[1]:
    missing_columns = set(dataset.columns) - set(testset_2023_2024.columns)
    for column in missing_columns:
        testset_2023_2024[column] = np.nan

In [ ]:
dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,16/08/03,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,M Halsey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/08/03,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,R Styles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16/08/03,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,J Winter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16/08/03,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,G Poll,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16/08/03,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,M Riley,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7596,28/05/2023,Everton,Bournemouth,1.0,0.0,H,0.0,0.0,D,S Attwell,...,2.14,-1.00,2.02,1.77,2.10,1.81,2.17,1.92,2.03,1.83
7597,28/05/2023,Leeds,Tottenham,1.0,4.0,A,0.0,1.0,A,A Taylor,...,2.50,0.25,1.84,2.06,1.83,2.10,1.90,2.14,1.81,2.06
7598,28/05/2023,Leicester,West Ham,2.0,1.0,H,1.0,0.0,H,S Hooper,...,2.51,-0.25,1.75,2.05,1.85,2.06,1.90,2.16,1.82,2.04
7599,28/05/2023,Man United,Fulham,2.0,1.0,H,1.0,1.0,D,R Jones,...,2.95,-1.25,1.98,1.92,1.98,1.93,2.07,1.98,1.97,1.89


In [ ]:
testset_2023_2024

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,LBH,BbMxAHH,GBD,SOD,LBA,BbAH,BbMx>2.5,SBA,B365AH,BbAvAHH
0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,02/03/2024,15:00,Tottenham,Crystal Palace,3,1,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,02/03/2024,17:30,Luton,Aston Villa,2,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,03/03/2024,13:00,Burnley,Bournemouth,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,03/03/2024,15:30,Man City,Man United,3,1,H,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# LabelEncoder Teams and Referee
le = LabelEncoder()
dataset["HomeTeam"] = le.fit_transform(dataset["HomeTeam"])
dataset["AwayTeam"] = le.fit_transform(dataset["AwayTeam"])
dataset["Referee"] = le.fit_transform(dataset["Referee"])

testset_2023_2024["HomeTeam"] = le.fit_transform(testset_2023_2024["HomeTeam"])
testset_2023_2024["AwayTeam"] = le.fit_transform(testset_2023_2024["AwayTeam"])
testset_2023_2024["Referee"] = le.fit_transform(testset_2023_2024["Referee"])

In [ ]:
# FTR and HTR: if HomeTeam wins (H) -> 1, else (D, A) -> 0
def only_hw(string):
    if string == 'H':
        return 1
    else:
        return 0

dataset['FTR'] = dataset.FTR.apply(only_hw)
dataset['HTR'] = dataset.HTR.apply(only_hw)

testset_2023_2024['FTR'] = testset_2023_2024.FTR.apply(only_hw)
testset_2023_2024['HTR'] = testset_2023_2024.HTR.apply(only_hw)

In [ ]:
# Impute NaN values
imputer = SimpleImputer(strategy="mean")
dataset = dataset.fillna(value=0)

testset_2023_2024 = testset_2023_2024.fillna(value=0)

In [ ]:
# Date object to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])
dataset['Date:year'] = dataset['Date'].dt.year
dataset['Date:month'] = dataset['Date'].dt.month
dataset['Date:day'] = dataset['Date'].dt.day
dataset['Date:day_of_week'] = dataset['Date'].dt.day_of_week
dataset['Date:day_of_year'] = dataset['Date'].dt.day_of_year
dataset[['Date','Date:year','Date:month','Date:day','Date:day_of_week','Date:day_of_year']].head()

testset_2023_2024["Date"] = pd.to_datetime(testset_2023_2024["Date"])
testset_2023_2024['Date:year'] = testset_2023_2024['Date'].dt.year
testset_2023_2024['Date:month'] = testset_2023_2024['Date'].dt.month
testset_2023_2024['Date:day'] = testset_2023_2024['Date'].dt.day
testset_2023_2024['Date:day_of_week'] = testset_2023_2024['Date'].dt.day_of_week
testset_2023_2024['Date:day_of_year'] = testset_2023_2024['Date'].dt.day_of_year
testset_2023_2024[['Date','Date:year','Date:month','Date:day','Date:day_of_week','Date:day_of_year']].head()

<ipython-input-11-5cdf2d212d2b>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dataset["Date"] = pd.to_datetime(dataset["Date"])
<ipython-input-11-5cdf2d212d2b>:10: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  testset_2023_2024["Date"] = pd.to_datetime(testset_2023_2024["Date"])


,Date,Date:year,Date:month,Date:day,Date:day_of_week,Date:day_of_year
0,2023-11-08,2023,11,8,2,312
1,2023-12-08,2023,12,8,4,342
2,2023-12-08,2023,12,8,4,342
3,2023-12-08,2023,12,8,4,342
4,2023-12-08,2023,12,8,4,342


In [ ]:
dataset.dtypes

Date                datetime64[ns]
HomeTeam                     int64
AwayTeam                     int64
FTHG                       float64
FTAG                       float64
                         ...      
Date:year                    int64
Date:month                   int64
Date:day                     int64
Date:day_of_week             int64
Date:day_of_year             int64
Length: 155, dtype: object

In [ ]:
def g(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('float64')
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

dataset = g(dataset)
testset_2023_2024 = g(testset_2023_2024)

In [ ]:
# Separate into feature set and target variable
#X_all = dataset.drop(['Date', 'FTR', 'Time'], 1)
X_all = dataset.drop(['Date', 'FTR', 'Time', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], 1)
y_all = dataset['FTR']

#X_testset_2023_2024 = testset_2023_2024.drop(['Date', 'FTR', 'Time'], 1)
X_testset_2023_2024 = testset_2023_2024.drop(['Date', 'FTR', 'Time', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], 1)
y_testset_2023_2024 = testset_2023_2024['FTR']

<ipython-input-14-057c4087154d>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_all = dataset.drop(['Date', 'FTR', 'Time', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], 1)
<ipython-input-14-057c4087154d>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_testset_2023_2024 = testset_2023_2024.drop(['Date', 'FTR', 'Time', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR'], 1)


In [ ]:
X_all

,HomeTeam,AwayTeam,Referee,HS,AS,HST,AST,HF,AF,HC,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Date:year,Date:month,Date:day,Date:day_of_week,Date:day_of_year
0,0.0,15.0,34.0,11.0,13.0,5.0,7.0,8.0,15.0,6.0,...,0.00,0.00,0.00,0.00,0.00,2003.0,8.0,16.0,5.0,228.0
1,2.0,36.0,55.0,10.0,15.0,5.0,7.0,20.0,27.0,1.0,...,0.00,0.00,0.00,0.00,0.00,2003.0,8.0,16.0,5.0,228.0
2,3.0,41.0,26.0,25.0,8.0,13.0,5.0,8.0,14.0,6.0,...,0.00,0.00,0.00,0.00,0.00,2003.0,8.0,16.0,5.0,228.0
3,16.0,24.0,17.0,17.0,8.0,9.0,5.0,18.0,16.0,7.0,...,0.00,0.00,0.00,0.00,0.00,2003.0,8.0,16.0,5.0,228.0
4,20.0,32.0,38.0,12.0,13.0,7.0,10.0,27.0,15.0,2.0,...,0.00,0.00,0.00,0.00,0.00,2003.0,8.0,16.0,5.0,228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7596,15.0,6.0,56.0,13.0,7.0,6.0,2.0,11.0,12.0,9.0,...,1.81,2.17,1.92,2.03,1.83,2023.0,5.0,28.0,6.0,148.0
7597,19.0,36.0,4.0,19.0,11.0,2.0,7.0,7.0,5.0,12.0,...,2.10,1.90,2.14,1.81,2.06,2023.0,5.0,28.0,6.0,148.0
7598,20.0,39.0,59.0,13.0,16.0,4.0,3.0,8.0,10.0,3.0,...,2.06,1.90,2.16,1.82,2.04,2023.0,5.0,28.0,6.0,148.0
7599,23.0,16.0,52.0,21.0,10.0,8.0,3.0,14.0,10.0,5.0,...,1.93,2.07,1.98,1.97,1.89,2023.0,5.0,28.0,6.0,148.0


In [ ]:
y_all

0       1.0
1       1.0
2       1.0
3       1.0
4       0.0
       ... 
7596    1.0
7597    0.0
7598    1.0
7599    1.0
7600    0.0
Name: FTR, Length: 7601, dtype: float32

In [ ]:
# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                    test_size = 0.2,
                                                    random_state = 2,
                                                    stratify = y_all)

# BUILD A MODEL NEURAL NETWORK

In [ ]:
model_1 = Sequential()

model_1.add(Dense(units=32, activation='relu'))
model_1.add(Dense(units=1, activation='sigmoid'))

model_1.compile(loss='binary_crossentropy', metrics=['accuracy'])

losses = model_1.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
190/190 [==============================] - 2s 5ms/step - loss: 2.7135 - accuracy: 0.5546 - val_loss: 3.2441 - val_accuracy: 0.5562
Epoch 2/50
190/190 [==============================] - 1s 3ms/step - loss: 1.9359 - accuracy: 0.5683 - val_loss: 2.7889 - val_accuracy: 0.4786
Epoch 3/50
190/190 [==============================] - 1s 3ms/step - loss: 1.7867 - accuracy: 0.5913 - val_loss: 1.9129 - val_accuracy: 0.5891
Epoch 4/50
190/190 [==============================] - 0s 2ms/step - loss: 1.7522 - accuracy: 0.5961 - val_loss: 2.3146 - val_accuracy: 0.5694
Epoch 5/50
190/190 [==============================] - 0s 2ms/step - loss: 1.7304 - accuracy: 0.5882 - val_loss: 1.0032 - val_accuracy: 0.6259
Epoch 6/50
190/190 [==============================] - 0s 2ms/step - loss: 1.6850 - accuracy: 0.5921 - val_loss: 0.7595 - val_accuracy: 0.6739
Epoch 7/50
190/190 [==============================] - 0s 2ms/step - loss: 1.6793 - accuracy: 0.5908 - val_loss: 1.5312 - val_accuracy: 0.5490
Epoch 

In [ ]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                4736      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4769 (18.63 KB)
Trainable params: 4769 (18.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
eval_result = model_1.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

48/48 [==============================] - 0s 1ms/step - loss: 1.0141 - accuracy: 0.6483
[test loss, test accuracy]: [1.0141398906707764, 0.6482577323913574]


In [ ]:
eval_result = model_1.evaluate(X_testset_2023_2024, y_testset_2023_2024)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 0s 2ms/step - loss: 1.7067 - accuracy: 0.6119
[test loss, test accuracy]: [1.706660270690918, 0.611940324306488]


# HYPERPARAMETER TUNING

In [ ]:
def model_builder(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers", min_value=1, max_value=5)):
    if counter == 0:
      model.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
              input_dim=X_train.shape[1]
          )
      )
    else:
      model.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"])
          )
      )
    counter+=1

  model.add(Dense(1,activation="sigmoid"))

  model.compile(loss="binary_crossentropy",
                metrics=["accuracy"])

  return model

In [ ]:
tuner = kt.BayesianOptimization(model_builder,
                                max_trials=30,
                                objective='val_accuracy',
                                directory='tuner_results',
                                project_name='my_tuner')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early])

Trial 30 Complete [00h 00m 41s]
val_accuracy: 0.718606173992157

Best val_accuracy So Far: 0.718606173992157
Total elapsed time: 00h 08m 05s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 56,
 'activation0': 'sigmoid',
 'units1': 32,
 'activation1': 'relu',
 'units2': 48,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 80,
 'activation4': 'relu'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                8288      
                                                                 
 dense_1 (Dense)             (None, 1)                 57        
                                                                 
Total params: 8345 (32.60 KB)
Trainable params: 8345 (32.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
190/190 [==============================] - 1s 5ms/step - loss: 0.5838 - accuracy: 0.7036 - val_loss: 0.5784 - val_accuracy: 0.7147
Epoch 2/50
190/190 [==============================] - 1s 4ms/step - loss: 0.5834 - accuracy: 0.7030 - val_loss: 0.5751 - val_accuracy: 0.7153
Epoch 3/50
190/190 [==============================] - 1s 4ms/step - loss: 0.5811 - accuracy: 0.7049 - val_loss: 0.5749 - val_accuracy: 0.7133
Epoch 4/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5874 - accuracy: 0.7000 - val_loss: 0.5950 - val_accuracy: 0.7087
Epoch 5/50
190/190 [==============================] - 0s 3ms/step - loss: 0.5864 - accuracy: 0.7000 - val_loss: 0.5768 - val_accuracy: 0.7028
Epoch 6/50
190/190 [==============================] - 0s 2ms/step - loss: 0.5872 - accuracy: 0.6959 - val_loss: 0.6140 - val_accuracy: 0.6548
Epoch 7/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5819 - accuracy: 0.7063 - val_loss: 0.5707 - val_accuracy: 0.7186
Epoch 

In [ ]:
eval_result = model.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

48/48 [==============================] - 0s 2ms/step - loss: 0.5906 - accuracy: 0.6897
[test loss, test accuracy]: [0.5905564427375793, 0.6896778345108032]


In [ ]:
eval_result = model.evaluate(X_testset_2023_2024, y_testset_2023_2024)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 0s 3ms/step - loss: 0.7115 - accuracy: 0.5560
[test loss, test accuracy]: [0.7114555239677429, 0.5559701323509216]


# REGULARIZATION (DROPOUT + L1 REGULARIZATION)

---



In [ ]:
def model_builder_2(hp):
  model_2 = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers", min_value=1, max_value=5)):
    if counter == 0:
      model_2.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
              input_dim=X_train.shape[1],
              kernel_regularizer=keras.regularizers.l1(hp.Float("l1_regularizer", min_value=0.0, max_value=0.01))
          )
      )
      model_2.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:
      model_2.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
              kernel_regularizer=keras.regularizers.l1(hp.Float("l1_regularizer", min_value=0.0, max_value=0.01))
          )
      )
      model_2.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    counter+=1

  model_2.add(Dense(1,activation="sigmoid"))

  model_2.compile(loss="binary_crossentropy",
                  metrics=["accuracy"])

  return model_2

In [ ]:
tuner_2 = kt.BayesianOptimization(model_builder_2,
                                max_trials=30,
                                objective='val_accuracy',
                                directory='tuner_results_2',
                                project_name='my_tuner_2')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner_2.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early])

Trial 30 Complete [00h 00m 10s]
val_accuracy: 0.5417488217353821

Best val_accuracy So Far: 0.7028270959854126
Total elapsed time: 00h 08m 19s


In [ ]:
tuner_2.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 40,
 'activation0': 'relu',
 'l1_regularizer': 0.0016599336390814934,
 'dropout0': 0.2,
 'units1': 104,
 'activation1': 'relu',
 'dropout1': 0.9,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.3,
 'units3': 128,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 120,
 'activation4': 'tanh',
 'dropout4': 0.5}

In [ ]:
model_2 = tuner_2.get_best_models(num_models=1)[0]

In [ ]:
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                5920      
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 41        
                                                                 
Total params: 5961 (23.29 KB)
Trainable params: 5961 (23.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_2.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6583 - accuracy: 0.6571 - val_loss: 0.6228 - val_accuracy: 0.6746
Epoch 2/50
190/190 [==============================] - 0s 3ms/step - loss: 0.6479 - accuracy: 0.6607 - val_loss: 0.8072 - val_accuracy: 0.6029
Epoch 3/50
190/190 [==============================] - 0s 3ms/step - loss: 0.6434 - accuracy: 0.6663 - val_loss: 0.6105 - val_accuracy: 0.6746
Epoch 4/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6447 - accuracy: 0.6617 - val_loss: 0.6128 - val_accuracy: 0.6700
Epoch 5/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6378 - accuracy: 0.6653 - val_loss: 0.6120 - val_accuracy: 0.6903
Epoch 6/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6501 - accuracy: 0.6681 - val_loss: 0.6831 - val_accuracy: 0.6246
Epoch 7/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6541 - accuracy: 0.6648 - val_loss: 0.6951 - val_accuracy: 0.5989
Epoch 

In [ ]:
eval_result = model_2.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

48/48 [==============================] - 0s 3ms/step - loss: 0.5958 - accuracy: 0.7015
[test loss, test accuracy]: [0.5957757234573364, 0.7015121579170227]


In [ ]:
eval_result = model_2.evaluate(X_testset_2023_2024, y_testset_2023_2024)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 0s 3ms/step - loss: 0.5730 - accuracy: 0.7164
[test loss, test accuracy]: [0.5729632377624512, 0.7164179086685181]


# OPTIMIZATION

In [ ]:
def model_builder_3(hp):
  model_3 = Sequential()

  counter = 0

  for i in range(hp.Int("num_layers", min_value=1, max_value=5)):
    if counter == 0:
      model_3.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
              input_dim=X_train.shape[1],
              kernel_regularizer=keras.regularizers.l1(hp.Float("l1_regularizer", min_value=0.0, max_value=0.01))
          )
      )
      model_3.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:
      model_3.add(
          Dense(
              hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
              kernel_regularizer=keras.regularizers.l1(hp.Float("l1_regularizer", min_value=0.0, max_value=0.01))
          )
      )
      model_3.add(Dropout(hp.Choice("dropout" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    counter+=1

  model_3.add(Dense(1,activation="sigmoid"))

  model_3.compile(optimizer=hp.Choice("optimizer", values=["rmsprop", "adam"]),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

  return model_3

In [ ]:
tuner_3 = kt.BayesianOptimization(model_builder_3,
                                max_trials=30,
                                objective='val_accuracy',
                                directory='tuner_results_3',
                                project_name='my_tuner_3')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner_3.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early])

Trial 30 Complete [00h 00m 16s]
val_accuracy: 0.5417488217353821

Best val_accuracy So Far: 0.7153188586235046
Total elapsed time: 00h 09m 25s


In [ ]:
tuner_3.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 96,
 'activation0': 'relu',
 'l1_regularizer': 0.0005972349581396186,
 'dropout0': 0.1,
 'optimizer': 'adam',
 'units1': 32,
 'activation1': 'tanh',
 'dropout1': 0.7,
 'units2': 104,
 'activation2': 'sigmoid',
 'dropout2': 0.5,
 'units3': 48,
 'activation3': 'sigmoid',
 'dropout3': 0.3,
 'units4': 24,
 'activation4': 'tanh',
 'dropout4': 0.5}

In [ ]:
model_3 = tuner_3.get_best_models(num_models=1)[0]

In [ ]:
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                14208     
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 97        
                                                                 
Total params: 14305 (55.88 KB)
Trainable params: 14305 (55.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_3.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
190/190 [==============================] - 1s 4ms/step - loss: 0.6048 - accuracy: 0.7008 - val_loss: 0.6221 - val_accuracy: 0.6851
Epoch 2/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6023 - accuracy: 0.6992 - val_loss: 0.6028 - val_accuracy: 0.6917
Epoch 3/50
190/190 [==============================] - 1s 3ms/step - loss: 0.6073 - accuracy: 0.6926 - val_loss: 0.5923 - val_accuracy: 0.7101
Epoch 4/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5967 - accuracy: 0.7021 - val_loss: 0.5929 - val_accuracy: 0.7009
Epoch 5/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5955 - accuracy: 0.7054 - val_loss: 0.5949 - val_accuracy: 0.7068
Epoch 6/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5971 - accuracy: 0.6987 - val_loss: 0.5852 - val_accuracy: 0.7166
Epoch 7/50
190/190 [==============================] - 1s 3ms/step - loss: 0.5943 - accuracy: 0.7003 - val_loss: 0.5853 - val_accuracy: 0.7186
Epoch 

In [ ]:
eval_result = model_3.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

48/48 [==============================] - 0s 2ms/step - loss: 0.5682 - accuracy: 0.7087
[test loss, test accuracy]: [0.5682243704795837, 0.7087442278862]


In [ ]:
eval_result = model_3.evaluate(X_testset_2023_2024, y_testset_2023_2024)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.6903
[test loss, test accuracy]: [0.5749714374542236, 0.6902984976768494]
